✅ Identify & drop highly correlated features

From your heatmap, examples of highly collinear pairs (r > 0.95):

spkts and sbytes

smean and spkts

stcpb and dtcpb

sinpkt and dmean

dbytes and dpkts

In [1]:
import pandas as pd

data = pd.read_parquet(r"Data\UNSW_NB15_training-set.parquet", engine="pyarrow")
print(data.head())

        dur proto service state  spkts  dpkts  sbytes  dbytes       rate  \
0  0.121478   tcp       -   FIN      6      4     258     172  74.087486   
1  0.649902   tcp       -   FIN     14     38     734   42014  78.473373   
2  1.623129   tcp       -   FIN      8     16     364   13186  14.170161   
3  1.681642   tcp     ftp   FIN     12     12     628     770  13.677108   
4  0.449454   tcp       -   FIN     10      6     534     268  33.373825   

          sload  ...  trans_depth  response_body_len  ct_src_dport_ltm  \
0  14158.942383  ...            0                  0                 1   
1   8395.112305  ...            0                  0                 1   
2   1572.271851  ...            0                  0                 1   
3   2740.178955  ...            0                  0                 1   
4   8561.499023  ...            0                  0                 2   

   ct_dst_sport_ltm  is_ftp_login  ct_ftp_cmd  ct_flw_http_mthd  \
0                 1            

In [2]:
#Drop one feature per pair:

high_corr_features = ['spkts', 'smean', 'dtcpb', 'dmean', 'dpkts']
data_reduced = data.drop(columns=high_corr_features)
data_reduced.columns


Index(['dur', 'proto', 'service', 'state', 'sbytes', 'dbytes', 'rate', 'sload',
       'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin',
       'stcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'trans_depth',
       'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports',
       'attack_cat', 'label'],
      dtype='object')

Encode categorical variables

Normalize or standardize numerical features

In [48]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data_encoded = data.copy()
for col in data_encoded.select_dtypes(include='category').columns:
    data_encoded[col] = LabelEncoder().fit_transform(data_encoded[col])


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

data_encoded = data.copy()
for col in data_encoded.select_dtypes(include='category').columns:
    data_encoded[col] = LabelEncoder().fit_transform(data_encoded[col])

# Repeat for df_reduced
scaler = StandardScaler()
X_full = scaler.fit_transform(data_encoded.drop(columns=['attack_cat', 'label']))
X_reduced = scaler.fit_transform(data_encoded.drop(columns=high_corr_features + ['attack_cat', 'label']))
y = data_encoded['label']


In [49]:
data_encoded

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,attack_cat,label
0,0.121478,113,0,2,6,4,258,172,74.087486,1.415894e+04,...,0,0,1,1,0,0,0,0,6,0
1,0.649902,113,0,2,14,38,734,42014,78.473373,8.395112e+03,...,0,0,1,1,0,0,0,0,6,0
2,1.623129,113,0,2,8,16,364,13186,14.170161,1.572272e+03,...,0,0,1,1,0,0,0,0,6,0
3,1.681642,113,3,2,12,12,628,770,13.677108,2.740179e+03,...,0,0,1,1,1,1,0,0,6,0
4,0.449454,113,0,2,10,6,534,268,33.373825,8.561499e+03,...,0,0,2,1,0,0,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.000009,119,2,3,2,0,114,0,111111.109375,5.066666e+07,...,0,0,24,13,0,0,0,0,5,1
175337,0.505762,113,0,2,10,8,620,354,33.612648,8.826286e+03,...,0,0,1,1,0,0,0,0,8,1
175338,0.000009,119,2,3,2,0,114,0,111111.109375,5.066666e+07,...,0,0,3,3,0,0,0,0,5,1
175339,0.000009,119,2,3,2,0,114,0,111111.109375,5.066666e+07,...,0,0,30,14,0,0,0,0,5,1


In [4]:
X_reduced.shape

(175341, 29)

In [5]:
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN

# Isolation Forest
iso = IsolationForest(contamination=0.05, random_state=42)
pred_iso = iso.fit_predict(X_full)  # -1 = anomaly, 1 = normal


In [6]:
# GMM
gmm = GaussianMixture(n_components=2, random_state=42)
gmm_labels = gmm.fit_predict(X_full)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10, random_state=42)
X_pca = pca.fit_transform(X_full)

dbscan = DBSCAN(eps=1.5, min_samples=5)
db_labels = dbscan.fit_predict(X_pca)


In [7]:
gmm_labels

array([0, 0, 0, ..., 1, 1, 1], shape=(175341,))

In [8]:
from sklearn.metrics import f1_score, adjusted_rand_score

f1 = f1_score(y, (pred_iso == -1).astype(int))
ari = adjusted_rand_score(y, (pred_iso == -1).astype(int))
print("Isolation Forest F1:", f1, "| ARI:", ari)


Isolation Forest F1: 0.08965872545040122 | ARI: 0.003743917028665791


In [9]:
from sklearn.metrics import f1_score, adjusted_rand_score

f1 = f1_score(y, gmm_labels.astype(int))
ari = adjusted_rand_score(y, gmm_labels.astype(int))
print("Gmm F1:", f1, "| ARI:", ari)


Gmm F1: 0.7283791995027622 | ARI: 0.1091433542732965


In [10]:
test_data = pd.read_parquet(r"Data\UNSW_NB15_testing-set.parquet", engine="pyarrow")
test_data.shape

(82332, 36)

In [11]:
test_data.columns

Index(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt',
       'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt',
       'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports',
       'attack_cat', 'label'],
      dtype='object')

In [12]:
Test_data_copy = test_data.copy()

In [13]:
for col in test_data.select_dtypes(include='category').columns:
    test_data[col] = LabelEncoder().fit_transform(test_data[col])
X_test = scaler.fit_transform(test_data.drop(columns=['attack_cat', 'label']))
y_test = test_data['label']

In [14]:
print(X_test.shape)
print(y_test.shape)

(82332, 34)
(82332,)


In [15]:
X_full.shape

(175341, 34)

In [16]:
y.shape

(175341,)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix


X_train = X_full
y_train = y
clf = RandomForestClassifier(class_weight="balanced")
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.99      0.52      0.68     37000
           1       0.72      1.00      0.83     45332

    accuracy                           0.78     82332
   macro avg       0.85      0.76      0.76     82332
weighted avg       0.84      0.78      0.76     82332



In [ ]:
support_vector = SVC(kernel='linear',random_state=42)
support_vector.fit(X_train, y_train)
pred = support_vector.predict(X_test)
print(classification_report(y_test, pred))

In [18]:
Knn = KNeighborsClassifier()
Knn.fit(X_train, y_train)
pred = Knn.predict(X_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.94      0.66      0.77     37000
           1       0.78      0.97      0.86     45332

    accuracy                           0.83     82332
   macro avg       0.86      0.81      0.82     82332
weighted avg       0.85      0.83      0.82     82332



In [19]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
pred = decision_tree.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.65      0.52      0.57     37000
           1       0.66      0.77      0.71     45332

    accuracy                           0.66     82332
   macro avg       0.65      0.64      0.64     82332
weighted avg       0.65      0.66      0.65     82332



Now let's fit knn, decsion trees, random forest and gmm (since they're performing quiet well) using dataset that we have removed some elements that had collinearlity

In [26]:
# GMM
gmm = GaussianMixture(n_components=2, random_state=42)
gmm_labels = gmm.fit_predict(X_reduced)

f1 = f1_score(y, gmm_labels.astype(int))
ari = adjusted_rand_score(y, gmm_labels.astype(int))
print("Gmm F1:", f1, "| ARI:", ari)


Gmm F1: 0.035621751104013555 | ARI: 0.0022309682063044673


In [20]:
X_test

array([[-0.21372745,  0.41056274, -0.6744059 , ..., -0.09061736,
        -0.20314282, -0.10607007],
       [-0.21372808,  0.41056274, -0.6744059 , ..., -0.09061736,
        -0.20314282, -0.10607007],
       [-0.21372872,  0.41056274, -0.6744059 , ..., -0.09061736,
        -0.20314282, -0.10607007],
       ...,
       [-0.21372978, -5.54527746, -0.6744059 , ..., -0.09061736,
        -0.20314282,  9.42773017],
       [-0.21372978, -5.54527746, -0.6744059 , ..., -0.09061736,
        -0.20314282,  9.42773017],
       [-0.21372787,  0.41056274, -0.6744059 , ..., -0.09061736,
        -0.20314282, -0.10607007]], shape=(82332, 34))

In [89]:
X_test = scaler.fit_transform(X_test)

In [21]:
X_test[:5]

array([[-0.21372745,  0.41056274, -0.6744059 ,  0.93269533, -0.1244551 ,
        -0.15181641, -0.04368361, -0.08736871,  0.05718099,  0.6439127 ,
        -0.26349797, -0.07353054, -0.11324391, -0.12217934, -0.09416902,
        -0.11217671, -0.14721835, -1.04791956, -0.77984004, -0.77675409,
        -1.00624379, -0.48202491, -0.41290971, -0.48407269,  0.52031938,
        -0.47537059, -0.17364821, -0.04190986, -0.46831162, -0.45018649,
        -0.09085748, -0.09061736, -0.20314282, -0.10607007],
       [-0.21372808,  0.41056274, -0.6744059 ,  0.93269533, -0.1244551 ,
        -0.15181641, -0.03630776, -0.08736871,  0.28656485,  4.53935073,
        -0.26349797, -0.07353054, -0.11324391, -0.12217982, -0.09416902,
        -0.11217671, -0.14721835, -1.04791956, -0.77984004, -0.77675409,
        -1.00624379, -0.48202491, -0.41290971, -0.48407269,  3.55671589,
        -0.47537059, -0.17364821, -0.04190986, -0.46831162, -0.45018649,
        -0.09085748, -0.09061736, -0.20314282, -0.10607007],
  

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_reduced, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))

In [36]:
Knn = KNeighborsClassifier()
Knn.fit(X_reduced, y_train)
pred = Knn.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.65      0.77     37000
           1       0.77      0.97      0.86     45332

    accuracy                           0.82     82332
   macro avg       0.85      0.81      0.81     82332
weighted avg       0.85      0.82      0.82     82332



since this dataset is imbalanced we'll try to handle the dataset using different techniques and try and increase the F1 score of the minority class

In [22]:
from imblearn.over_sampling import SMOTE
sm  = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_full, y)

In [23]:
print('After OverSampling, the shape of train_X: {}'.format(X_train.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train == 0)))

After OverSampling, the shape of train_X: (238682, 34)
After OverSampling, the shape of train_y: (238682,) 

After OverSampling, counts of label '1': 119341
After OverSampling, counts of label '0': 119341


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix



In [25]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
pred = decision_tree.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.61      0.86      0.71     37000
           1       0.83      0.56      0.67     45332

    accuracy                           0.69     82332
   macro avg       0.72      0.71      0.69     82332
weighted avg       0.73      0.69      0.69     82332



In [26]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.75      0.91      0.82     37000
           1       0.91      0.75      0.82     45332

    accuracy                           0.82     82332
   macro avg       0.83      0.83      0.82     82332
weighted avg       0.84      0.82      0.82     82332



In [27]:
Knn = KNeighborsClassifier()
Knn.fit(X_train, y_train)
pred = Knn.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.75      0.82     37000
           1       0.82      0.93      0.87     45332

    accuracy                           0.85     82332
   macro avg       0.86      0.84      0.84     82332
weighted avg       0.85      0.85      0.85     82332



At this point I decided to go with knn classifier and also random forest. There was not much I could do to the data to improve the metrics. Let's try fine tuning the parameteres using random searchCV and see if we can improve the metrics by 2-3%

In [28]:
#view the parameters of the current model
Knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [29]:
#view the parameters of the model
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

We specify different parameters for each of our selected model

In [40]:
knn_param_grid = {
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40],
    'n_neighbors': [2,3,5,6],
    'weights': ['uniform', 'distance']  
}

clf_param_grid = {
    'bootstrap': [True, False],
    'min_samples_split': [2,4,5,6],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt','log2', None],
    'n_estimators': [10,20, 40, 50, 80,100],
    'oob_score': [True, False],
    }

In [31]:
from sklearn.utils import resample
X_resample, y_resample = resample(X_train, y_train, n_samples=50000, random_state=42)

In [44]:
from sklearn.model_selection import RandomizedSearchCV

Knn_random_search = RandomizedSearchCV(
    Knn,
    knn_param_grid,
    cv = 3,
    n_jobs=-1,
    random_state=42   
)
Knn_random_search.fit(X_resample, y_resample)

# Print the best hyperparameters found by each method 
print(f"Best hyperparameters for knn found by RandomizedSearchCV: {Knn_random_search.best_params_}") 

Best hyperparameters for knn found by RandomizedSearchCV: {'weights': 'distance', 'n_neighbors': 2, 'leaf_size': 30, 'algorithm': 'kd_tree'}


In [41]:
from sklearn.model_selection import RandomizedSearchCV

clf_random_search = RandomizedSearchCV(
    clf,
    clf_param_grid, 
    cv=5,
    random_state=42,
    n_jobs=-1
)
clf_random_search.fit(X_resample, y_resample)

# Print the best hyperparameters found by each method 
print(f"Best hyperparameters for knn found by RandomizedSearchCV: {clf_random_search.best_params_}") 

c:\Users\ADMIN\Desktop\Login_Anomaly_Detection\venv\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ADMIN\Desktop\Login_Anomaly_Detection\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ADMIN\Desktop\Login_Anomaly_Detection\venv\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\ADMIN\Desktop\Login_Ano

Best hyperparameters for knn found by RandomizedSearchCV: {'oob_score': False, 'n_estimators': 100, 'min_samples_split': 2, 'max_features': 'log2', 'criterion': 'entropy', 'bootstrap': False}


Now I get to test out the parameters and see if they improve the overall model

In [45]:
#{'weights': 'distance', 'n_neighbors': 2, 'leaf_size': 30, 'algorithm': 'kd_tree'}
Tuned_Knn = KNeighborsClassifier(
    weights="distance",
    algorithm="kd_tree",
    n_neighbors=2,
    leaf_size= 30
)
Tuned_Knn.fit(X_train, y_train)
pred = Tuned_Knn.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.72      0.80     37000
           1       0.80      0.93      0.86     45332

    accuracy                           0.84     82332
   macro avg       0.85      0.83      0.83     82332
weighted avg       0.84      0.84      0.83     82332



In [46]:
#{'oob_score': False, 'n_estimators': 100, 'min_samples_split': 2, 'max_features': 'log2', 'criterion': 'entropy', 'bootstrap': False}
Tuned_clf = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=2,
    max_features="log2",
    criterion="entropy",
    bootstrap=False,
    oob_score= False
)
Tuned_clf.fit(X_train, y_train)
pred = Tuned_clf.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82     37000
           1       0.86      0.84      0.85     45332

    accuracy                           0.84     82332
   macro avg       0.84      0.84      0.84     82332
weighted avg       0.84      0.84      0.84     82332



After fine tuning, random forest increased by 2% which is a great success. Although Knn improved on the recall of the minority class, the accuracy decreased by 1% which is not a bad thing as long as the model does not overfit. At this point we shall save bost the fine tuned model as the f1 score and recall improved in bost cases.

In [51]:
Knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [52]:
Tuned_clf.get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [53]:
import joblib

joblib.dump(le, 'label_encoder.joblib')
joblib.dump(scaler, 'Scaler.joblib')

joblib.dump(Knn, 'KNN_Classifier.joblib')
joblib.dump(Tuned_clf, 'Random_forest.joblib')

['Random_forest.joblib']